In [16]:
# Standard libraries for manipulation of data
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

# pytorch related
import torch

# nflows related - nflows is a comprehensive collection of normalizing flows using PyTorch.
import nflows

# set random seed for numpy
np.random.seed(100)
# Checks if GPU is available or not
print(torch.cuda.is_available())

False


In [17]:
print(torch.cuda.current_device())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx